In [12]:
import boto3
import csv
import json
import random

In [19]:
# path = 'C:/Users/jordi/Documents/claus/'
path = '/home/propietari/Documents/claus/'
filename = 'AWS_S3_keys_wri.json'
file = path + filename
with open(file, 'r') as dict:
    credentials = json.load(dict)
                                      
KEY = list(credentials)[0]
SECRET = list(credentials.values())[0]
# s3BucketName = "wri-testing"
s3BucketName = "wri-nlp-policy"
# region = 'eu-central-1'
region = "us-east-1"

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

In [39]:
language = "english"
path = "../out/"
file_name = f"incentives_{language}.csv"
file = path + file_name

with open(file, newline='') as f:
    reader = csv.reader(f)
    next(reader)
    file_code = [row[0] for row in reader]

codes = {}
for item in random.sample(file_code, 50):
    codes[item] = 0

In [40]:
in_prefix = f"{language}_documents/HSSC/updated_sentences/"  

i = 0
documents_with_incentives = 0
results_list = [["Sentence_name", "Sentence text", "Label"]]

incentives = 0
for obj in s3.Bucket(s3BucketName).objects.all().filter(Prefix = in_prefix):
    name = obj.key.split("/")[3].split("_")[0]
#     print(name)
    if name in codes :
        sentences = json.loads(obj.get()['Body'].read().decode('utf-8'))
        for sentence_id in sentences[name]["sentences"]:
            incentive = sentences[name]["sentences"][sentence_id]["label"]
            if len(incentive) > 1 and incentive[0] == "Incentive":
                incentives += 1
                results_list.append([])
                results_list[incentives].append(name)
                results_list[incentives].append(sentences[name]["sentences"][sentence_id]["text"])
                results_list[incentives].append(incentive[1])
                
with open(f"../out/incentive_sentences_{language}.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(results_list)           

In [37]:
len(results_list)

408

In [38]:
with open(f"../out/incentive_sentences_{language}.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(results_list)           